## IMPORTING MODULES

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-2025/sample_submission.csv


## PACKING WHILE ENSURING NO COLLISION

In [2]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import random

# ============================================================
# Reproducibility
# ============================================================
np.random.seed(42)
random.seed(42)

# ============================================================
# Tunable constants
# ============================================================
REPEL_ITERS = 6
SOLVE_ITERS = 120
ROT_STEP = 2
FORCE_C = 0.001
MIN_DIST = 1e-4

# ============================================================
# Approximate polygon geometry of trees
# ============================================================
TREE_POLYGON = Polygon([
    (-0.35, 0.0),
    (-0.15, 0.6),
    (-0.25, 0.6),
    (0.0, 1.2),
    (0.25, 0.6),
    (0.15, 0.6),
    (0.35, 0.0),
])

TREE_RADIUS = TREE_POLYGON.bounds[2] - TREE_POLYGON.bounds[0]
TREE_H = TREE_POLYGON.bounds[3] - TREE_POLYGON.bounds[1]

# ============================================================
# Repulsion Physics
# ============================================================
def repel(xy, n):
    OUT = xy.copy()

    for _ in range(REPEL_ITERS):
        fx = np.zeros(n)
        fy = np.zeros(n)

        for i in range(n):
            for j in range(i + 1, n):
                dx = OUT[i, 0] - OUT[j, 0]
                dy = OUT[i, 1] - OUT[j, 1]
                dist = dx * dx + dy * dy

                if dist < MIN_DIST:
                    dist = MIN_DIST

                f = FORCE_C / dist
                fx[i] += f * dx
                fy[i] += f * dy
                fx[j] -= f * dx
                fy[j] -= f * dy

        OUT[:, 0] += fx
        OUT[:, 1] += fy

    return OUT

# ============================================================
# Hexagonal initialization
# ============================================================
def hex_init(n, radius):
    cols = int(np.ceil(np.sqrt(n)))
    rows = cols
    pts = []
    k = 0

    for r in range(rows):
        for c in range(cols):
            if k >= n:
                break
            x = (c - cols / 2) * radius * 1.1 + (r % 2) * radius * 0.55
            y = (r - rows / 2) * radius * 0.95
            pts.append((x, y))
            k += 1

    return np.array(pts[:n])

# ============================================================
# Collision check
# ============================================================
def collide(xy, rot, n):
    polys = []

    for i in range(n):
        p = rotate(TREE_POLYGON, rot[i], origin=(0, 0))
        p = translate(p, xy[i, 0], xy[i, 1])

        for q in polys:
            if p.intersects(q):
                return True

        polys.append(p)

    return False

# ============================================================
# Bounding box side length
# ============================================================
def side_len(xy, rot, n):
    X, Y = [], []

    for i in range(n):
        p = rotate(TREE_POLYGON, rot[i], origin=(0, 0))
        p = translate(p, xy[i, 0], xy[i, 1])
        bx = p.bounds
        X.extend([bx[0], bx[2]])
        Y.extend([bx[1], bx[3]])

    return max(max(X) - min(X), max(Y) - min(Y))

# ============================================================
# Solver (SAFE VERSION)
# ============================================================
def solve_n(n):
    radius = max(TREE_RADIUS, TREE_H) * 0.65

    def fallback():
        xy = hex_init(n, radius * 2.2)
        rot = np.zeros(n)
        return xy, rot

    if n == 1:
        return np.array([[0.0, 0.0]]), np.array([0.0])

    xy = hex_init(n, radius)
    rot = np.random.uniform(0, 360, n)

    best_xy = xy.copy()
    best_rot = rot.copy()
    best_s = float("inf")

    for _ in range(SOLVE_ITERS):
        xy = repel(xy, n)

        rid = random.randint(0, n - 1)
        rot[rid] += random.uniform(-ROT_STEP, ROT_STEP)

        if collide(xy, rot, n):
            continue

        s = side_len(xy, rot, n)
        if s < best_s:
            best_s = s
            best_xy = xy.copy()
            best_rot = rot.copy()

    if collide(best_xy, best_rot, n):
        return fallback()

    return best_xy, best_rot

# ============================================================
# Main loop → submission
# ============================================================
rows = []

for n in range(1, 201):
    print("Packing", n)
    xy, rot = solve_n(n)

    for i in range(n):
        rows.append([
            f"{n:03d}_{i}",
            "s" + str(round(float(xy[i, 0]), 6)),
            "s" + str(round(float(xy[i, 1]), 6)),
            "s" + str(round(float(rot[i]), 6))
        ])

df = pd.DataFrame(rows, columns=["id", "x", "y", "deg"])
df.to_csv("submission.csv", index=False)

print("Done!")


Packing 1
Packing 2
Packing 3
Packing 4
Packing 5
Packing 6
Packing 7
Packing 8
Packing 9
Packing 10
Packing 11
Packing 12
Packing 13
Packing 14
Packing 15
Packing 16
Packing 17
Packing 18
Packing 19
Packing 20
Packing 21
Packing 22
Packing 23
Packing 24
Packing 25
Packing 26
Packing 27
Packing 28
Packing 29
Packing 30
Packing 31
Packing 32
Packing 33
Packing 34
Packing 35
Packing 36
Packing 37
Packing 38
Packing 39
Packing 40
Packing 41
Packing 42
Packing 43
Packing 44
Packing 45
Packing 46
Packing 47
Packing 48
Packing 49
Packing 50
Packing 51
Packing 52
Packing 53
Packing 54
Packing 55
Packing 56
Packing 57
Packing 58
Packing 59
Packing 60
Packing 61
Packing 62
Packing 63
Packing 64
Packing 65
Packing 66
Packing 67
Packing 68
Packing 69
Packing 70
Packing 71
Packing 72
Packing 73
Packing 74
Packing 75
Packing 76
Packing 77
Packing 78
Packing 79
Packing 80
Packing 81
Packing 82
Packing 83
Packing 84
Packing 85
Packing 86
Packing 87
Packing 88
Packing 89
Packing 90
Packing 91
Packing 

## PLEASE UPVOTE MY WORK